In [507]:
import tifffile
import numpy as np
import os
import pandas as pd
import re
from glob import glob
import matplotlib.pyplot as plt
from skimage.filters import gaussian
from skimage.transform import rotate
from scipy.signal import find_peaks
from ipywidgets import interactive, interact
from tqdm.notebook import tqdm
from pystackreg import StackReg
import os
from functools import lru_cache
import warnings
from joblib import Parallel, delayed

In [518]:
class Experiment:
    def __init__(self, directory):
        self.directory = directory
        self.extracted_dir = directory + "/extracted/"
        self.files = glob(extracted_dir+"*")
        self.FOVs = [x.split("/")[-1].split("_")[0] for x in files]
        self.FOVs = sorted(list(set(self.FOVs)))
        self.times = [x.split("/")[-1].split("_")[1] for x in files]
        self.times = sorted(list(set(self.times)))
        self.channels = [x.split("/")[-1].split("_")[2].split(".")[0] for x in files]
        self.channels = sorted(list(set(self.channels)))
        self.file_extension = files[0].split("/")[-1].split("_")[2].split(".")[-1]
        
    def img_path_generator(self, directory, FOV, time, channel, file_extension):
        return f"{directory}/{FOV}_{time}_{channel}.{file_extension}" 
    
    @property
    def mean_start(self):
        return self._mean_start
    
    @mean_start.setter
    def mean_start(self, value):
        assert value in ["START", "END"], "mean_start should be START or END"
        self._mean_start = value
    
    @property
    def registration_channel(self):
        return self._registration_channel
    
    @registration_channel.setter
    def registration_channel(self,value):
        assert value in self.channels, f"Channel must be one of {self.channels}"
        self._registration_channel = value
    
    
    def get_mean_images(self, rotation=0):
        if self._mean_start == "END":
            mean_times = self.times[-15:]
        elif self._mean_start == "START":
            mean_times = self.times[:15]
        
        ex_path = self.img_path_generator(self.extracted_dir,self.FOVs[0],self.times[0],self.channels[1],self.file_extension)
        img_size = tifffile.imread(ex_path).shape
        mean_images = dict()
        for FOV in self.FOVs:
            mean_img = np.zeros(img_size)
            mean_img_paths = [
                self.img_path_generator(
                    self.extracted_dir,FOV,x,self._registration_channel,file_extension) for x in mean_times
            ]
            mean_img_imgs = (tifffile.imread(x) for x in mean_img_paths)

            for img in mean_img_imgs:
                mean_img += img/mean_amount
            mean_img = rotate(mean_img, rotation)
            mean_images[FOV] = mean_img.astype(np.uint16)
        print(f"Mean images for {len(self.FOVs)} FOVs with rotation of {rotation}deg calculated, use the mean_images method to return a dict of mean images")
        self.mean_images = mean_images
    
    def get_image(self, FOV, time, channel):
        directory = img_path_generator(self.extracted_dir, FOV, time, channel, self.file_extension)
        return tifffile.imread(directory)
    
    def register_experiment(self, force=False):
        try:
            os.mkdir(directory+"/registered/")
        except:
            if force:
                warnings.warn("Reregistering experiment")
            elif not force:
                raise Exception("The experiment has already been registered, to re-register use force=True")
        self.registered_dir = directory+"/registered/"
        Parallel(n_jobs=-1)(delayed(self.register_FOV)(FOV) for FOV in self.FOVs)
        
    def register_FOV(self, FOV):
        ref = self.mean_images[FOV]
        for time in self.times:
            img_path = img_path_generator(self.extracted_dir, FOV, time, self._registration_channel, self.file_extension)
            mov = tifffile.imread(img_path)
            sr = StackReg(StackReg.RIGID_BODY)
            sr.register(ref, mov)
            mov = sr.transform(mov)
            mov = mov.astype(np.uint16)
            out_path = img_path_generator(self.registered_dir, FOV, time, self._registration_channel, self.file_extension)
            tifffile.imwrite(out_path, mov)
            for channel in self.channels:
                if channel == self._registration_channel:
                    pass
                else:
                    img_path = img_path_generator(self.extracted_dir, FOV, time, channel, self.file_extension)
                    mov = tifffile.imread(img_path)
                    mov = sr.transform(mov)
                    mov = mov.astype(np.uint16)
                    out_path = img_path_generator(self.registered_dir, FOV, time, channel, self.file_extension)
                    tifffile.imwrite(out_path, mov)
            

In [519]:
directory = '/home/georgeos/Storage/Dropbox/Dropbox (Cambridge University)/Nikon Nd2 Backup_Bakshi-Lab/sb2330/20220331/tna_GFP_SB6_exit/'

experiment = Experiment(directory)

experiment.mean_start = "END"
print(experiment.channels)

['Green', 'PC', 'Red', 'YFP']


In [520]:
experiment.registration_channel = "PC"
experiment.get_mean_images(rotation=-1)

Mean images for 10 FOVs with rotation of -1deg calculated, use the mean_images method to return a dict of mean images


In [ ]:
experiment.register_experiment(force=True)

<ipython-input-518-afdadfbecd37>:69: UserWarning: Reregistering experiment
  warnings.warn("Reregistering experiment")
